In [ ]:
import numpy as np
import pandas as pd

# 데이터 로드 - 판다스

In [ ]:
train_path = '/content/drive/MyDrive/2022-1/국비지원교육/실습폴더/ratings_train.txt'
with open(train_path, 'r') as f:
    train = f.readlines()

train_li = []
for a in train:
    a = a.split('\t')
    a[-1] = a[-1][0]
    train_li.append(a)

train = pd.DataFrame(data=train_li[1:], columns = train_li[0])
x_train = train[['id','document']]
y_train = train[['l']]

test_path = '/content/drive/MyDrive/2022-1/국비지원교육/실습폴더/ratings_test.txt'
with open(test_path, 'r') as f:
    test = f.readlines()

test_li = []
for a in test:
    a = a.split('\t')
    a[-1] = a[-1][0]
    test_li.append(a)

test = pd.DataFrame(data=test_li[1:], columns = test_li[0])
x_test = test[['id','document']]
y_test = test[['l']]

y_train['l'] = list(map(int,y_train['l']))
y_test['l'] = list(map(int,y_test['l']))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
train

,id,document,l
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


# Kiwipiepy 형태소 분석기

In [ ]:
# !pip install --upgrade pip
# !pip install kiwipiepy

     |████████████████████████████████| 2.1 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.6/584.6 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.7/25.7 MB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.10.0-py3-none-any.whl size=25882655 sha256=e9c84a039cf9fe4ca2050e00f3d0ba5debcfa748df7e1ef03ab4a2a3c2374f29
  Stored in directory: /root/.cache/pip/wheels/42/2f/c9/c82156eb15ed5ebb5475002723d1ab319947aca25bd663cd1d
Successfully built kiwipiepy-model


In [ ]:
# !python3 -m kiwipiepy

In [ ]:
from kiwipiepy import Kiwi, Option
from kiwipiepy.utils import Stopwords

# 키위파이 설명

In [ ]:
# 키위파이 링크 https://github.com/bab2min/kiwipiepy


# Kiwi(num_workers=0, model_path=None, options=Option.LOAD_DEFAULT_DICTIONARY | Option.INTEGRATE_ALLOMORPH, load_default_dict=True, integrate_allomorph=True)


kiwi = Kiwi(options = Option.INTEGRATE_ALLOMORPH) # 아/어/었 등 음운 통합
stopwords = Stopwords()
# stopwords.add(('10','SN'))

In [ ]:
tokens=kiwi.tokenize(x_train.document[10], stopwords = stopwords,
                     normalize_coda=True) # 이상한 받침 제거

tokens_list=[t.form for t in tokens if t.tag in {'MM','NNG','NP','NNP','VV','VA','VX','IC','MAG','MAJ','MM'}]

In [ ]:
tokens_list

['걍인피니트가짱', '진짜', '짱']

In [ ]:
sent_tokens = kiwi.split_into_sents(x_train.document[10],return_tokens=False) # 문장분리
tokens_list=[]
for t in sent_tokens:
    tokens_list.append(t.text)

tokens_list

['걍인피니트가짱이다.', '진짜짱이다♥']

# 트레인 데이터 토큰화

In [ ]:
kiwi = Kiwi()
stopwords = Stopwords()
train_tokens = []
for sentence in x_train.document:
    tokens = kiwi.tokenize(sentence,stopwords=stopwords,normalize_coda=True)
    temp = [t.form for t in tokens if t.tag[0] in {'M','N','V','I'}]
    train_tokens.append(temp)
x_train['tokens'] = np.array(train_tokens)
train_labels = np.array(y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


# 테스트데이터 토큰화

In [ ]:
kiwi = Kiwi()
stopwords = Stopwords()
test_tokens = []
for sentence in x_test.document:
    tokens = kiwi.tokenize(sentence,stopwords=stopwords,normalize_coda=True)
    temp = [t.form for t in tokens if t.tag[0] in {'M','N','V','I'}]
    test_tokens.append(temp)

x_test['tokens'] = np.array(test_tokens)
test_labels = np.array(y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':


In [ ]:
# train_tok = pd.concat((x_train,y_train), axis=1)
# test_tok = pd.concat((x_test,y_test),axis=1)

# train_tok.to_excel('C:\\Users\\g8428\\train_data.xlsx',index=False)
# test_tok.to_excel('C:\\Users\\g8428\\test_data.xlsx',index=False)

# 텐서플로, 시퀀스화, 패딩

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPool1D, Dense, Dropout, Embedding, Flatten ,Input,GlobalAveragePooling1D

# 모델 생성

In [ ]:
# 훈련시작

vocab_size = 25000
embedding_dim = 32 # 각 단어 하나당 임베딩 차원
max_length = 100 # 단어 개수 = 인풋 디멘션
trunc_type = 'post' # 뒤에서 자르기
oov_tok = '<oov>' # oov 처리
num_filters = 100


In [ ]:
model = Sequential()
# 원핫인코딩 사이즈, 원핫벡터를 덴스벡터로 만들때의 차원수
# 단어 수가 10000개 원핫 -> 16차원에 임베딩 시키는 것
# model.add(Embedding(vocab_size, embedding_dim))
model.add(Embedding(input_dim = vocab_size,output_dim = embedding_dim))


# 히든계층 - 풀링, 활성함수
model.add(GlobalAveragePooling1D())
model.add(Dense(6, activation='relu'))

# 아웃풋 계층 - 시그모이드, 크로스엔트로피
model.add(Dense(1, activation='sigmoid')) # 감성분석 이진분류를 위해 시그모이드

model.compile(loss = 'binary_crossentropy', optimizer = 'adam',
              metrics=['accuracy'])

model.summary()

NameError: ignored

In [ ]:
# 토크나이저
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train.tokens)
# vocab 확인, 시퀀스화, 패딩
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_train.tokens)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [ ]:
padded.shape

(150000, 100)

In [ ]:
# 테스트 데이터도 시퀀스로
test_sequences = tokenizer.texts_to_sequences(x_test.tokens)
test_padd = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)

In [ ]:
inputs = Input(shape=(max_length))

# 100열의 행이 들어감 -> input_dim은 그 벡터의 값의 차원(총 단어수) = vocab_size
# output dim 은 그 vocab_size 차원의 벡터를 output dim 차원으로 축소하는 것
embedd = Embedding(input_dim = vocab_size,output_dim = embedding_dim)(inputs)

conv1 = Conv1D(filters=1, kernel_size=3, padding='same', activation='relu')(embedd)
# pool1 = MaxPool1D()(conv1)

conv2 = Conv1D(filters=1, kernel_size=4, padding='same', activation='relu')(embedd)
# pool2 = MaxPool1D()(conv2)

conv3 = Conv1D(filters=1, kernel_size=5, padding='same', activation='relu')(embedd)
# pool3 = MaxPool1D()(conv3)

concat = tf.concat((conv1, conv2, conv3), 1)
pool = MaxPool1D()(concat)

flatten = Flatten()(pool)

dense4 = Dense(6, activation='relu')(flatten)
dropout4 = Dropout(rate=0.2)(dense4)
outputs= Dense(1, activation='sigmoid')(dropout4)

model = Model(inputs=inputs, outputs=outputs)

model.compile(loss = 'binary_crossentropy', optimizer = 'adam',
              metrics=['accuracy'])
model.summary()

NameError: ignored

In [ ]:
num_epochs = 10
batch_size = 200

model.fit(padded[:2000],train_labels[:2000],
          epochs = num_epochs, batch_size=batch_size)

Epoch 1/10
10/10 [==============================] - 1s 10ms/step - loss: 0.6930 - accuracy: 0.5095
Epoch 2/10
10/10 [==============================] - 0s 10ms/step - loss: 0.6886 - accuracy: 0.5785
Epoch 3/10
10/10 [==============================] - 0s 10ms/step - loss: 0.6818 - accuracy: 0.6130
Epoch 4/10
10/10 [==============================] - 0s 10ms/step - loss: 0.6713 - accuracy: 0.7230
Epoch 5/10
10/10 [==============================] - 0s 10ms/step - loss: 0.6571 - accuracy: 0.7315
Epoch 6/10
10/10 [==============================] - 0s 10ms/step - loss: 0.6354 - accuracy: 0.7945
Epoch 7/10
10/10 [==============================] - 0s 10ms/step - loss: 0.6067 - accuracy: 0.7920
Epoch 8/10
10/10 [==============================] - 0s 10ms/step - loss: 0.5706 - accuracy: 0.8470
Epoch 9/10
10/10 [==============================] - 0s 10ms/step - loss: 0.5269 - accuracy: 0.8540
Epoch 10/10
10/10 [==============================] - 0s 10ms/step - loss: 0.4691 - accuracy: 0.8900


In [ ]:
model.predict_generator(test_padd[:10])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


array([[0.95583904],
       [0.47117043],
       [0.38781404],
       [0.06568865],
       [0.14755413],
       [0.9840396 ],
       [0.01431561],
       [0.214569  ],
       [0.12705949],
       [0.98227435]], dtype=float32)

# 모델 학습

In [ ]:
!rm '/content/drive/MyDrive/2022-1/국비지원교육/실습폴더/models/'*

In [ ]:
MODEL_SAVE_FOLDER = '/content/drive/MyDrive/2022-1/국비지원교육/실습폴더/models/'
model_path = f"{MODEL_SAVE_FOLDER}mnist-{{epoch:d}}-{{val_loss:.5f}}-{{val_accuracy:.5f}}.hdf5"

cb_checkpoint = ModelCheckpoint(filepath=model_path,
                                monitor='val_accuracy',
                                verbose=1,
                                save_weights_only=True,
                                save_best_only=True)
cb_early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

In [ ]:
num_epochs = 100
batch_size = 1500
valid_split = 0.1
hist = model.fit(padded, train_labels, validation_data=(test_padd,test_labels),
                 validation_split=valid_split,
                 epochs = num_epochs, batch_size = batch_size,
                 callbacks=[cb_early_stopping, cb_checkpoint])

Epoch 1/100
89/90 [============================>.] - ETA: 0s - loss: 0.5246 - accuracy: 0.7491
Epoch 1: val_accuracy improved from -inf to 0.80487, saving model to /content/drive/MyDrive/2022-1/국비지원교육/실습폴더/models/mnist-1-0.43042-0.80487.hdf5
90/90 [==============================] - 4s 32ms/step - loss: 0.5238 - accuracy: 0.7497 - val_loss: 0.4304 - val_accuracy: 0.8049
Epoch 2/100
89/90 [============================>.] - ETA: 0s - loss: 0.4043 - accuracy: 0.8250
Epoch 2: val_accuracy improved from 0.80487 to 0.81920, saving model to /content/drive/MyDrive/2022-1/국비지원교육/실습폴더/models/mnist-2-0.40221-0.81920.hdf5
90/90 [==============================] - 3s 29ms/step - loss: 0.4038 - accuracy: 0.8253 - val_loss: 0.4022 - val_accuracy: 0.8192
Epoch 3/100
89/90 [============================>.] - ETA: 0s - loss: 0.3639 - accuracy: 0.8494
Epoch 3: val_accuracy improved from 0.81920 to 0.82087, saving model to /content/drive/MyDrive/2022-1/국비지원교육/실습포

In [ ]:
model.

array([[0.95583904],
       [0.47117043],
       [0.38781404],
       [0.06568865],
       [0.14755413],
       [0.9840396 ],
       [0.01431561],
       [0.214569  ],
       [0.12705949],
       [0.98227435]], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train[['document']],train[['l']], test_size=0.2, random_state=42)